In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
import common as cm

# Part 1: Information Gain

Important note: this exercise uses Pandas (for data manipulation and analysis) and Graphviz (for graph-drawing) libraries. 

This exercise consists of 3 parts. Complete the first part to get a mark of 3.0, the first two parts to get 4.0, complete all assignments to get 5.0. 

1.1 ) There are 10 objects (data) characterized with 5 binary attributes:

In [2]:
attributeNames = ["attr 1", "attr 2", "attr 3", "attr 4", "attr 5"]

data = pd.DataFrame(
    [
        [1, 0, 1, 1, 1],
        [1, 1, 0, 0, 1],
        [0, 1, 1, 1, 1],
        [1, 0, 1, 0, 1],
        [1, 0, 0, 1, 1],
        [0, 0, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 0, 0, 1, 1],
        [0, 1, 0, 0, 1],
        [0, 0, 0, 1, 1],
    ],
    columns=attributeNames,
)


1.2) Each object is assigned to either a class "0" or "1". The assignments are as follows (cl):

In [3]:
data["cl"] = [1, 1, 1, 0, 0, 1, 1, 1, 0, 0]

Hint: How one can read data (columns) in Pandas:

In [4]:
# print(data["cl"])
# print(list(data["cl"]))
# print(set(data["cl"]))
print(list(data["attr 1"]))

[1, 1, 0, 1, 1, 0, 1, 1, 0, 0]


Hint: How split data (Pandas DataFrame) base on column:

In [5]:
data[data['cl']==0]

,attr 1,attr 2,attr 3,attr 4,attr 5,cl
3,1,0,1,0,1,0
4,1,0,0,1,1,0
8,0,1,0,0,1,0
9,0,0,0,1,1,0


Hint: How take values from column (Pandas Series):

In [6]:
for id, row in data['cl'].items():
    print(id,row)

0 1
1 1
2 1
3 0
4 0
5 1
6 1
7 1
8 0
9 0


1.3 )  Finish the below function for calculating entropy. $H(CL) = - \sum_{y \in CL}p(y)log_2p(y)$ It should return a value of entropy for an input vector CL. Assume that $log_2(0)$ is equal to 0.

In [7]:
def getEntropy(cl):
    pcl_0 = list(cl).count(0)/len(cl)
    pcl_1 = 1 - pcl_0
    E = -pcl_0*(math.log(pcl_0,2))-pcl_1*(math.log(pcl_1,2))
    return E

1.4 ) Calculate the entropy for the CL vector:

In [8]:
cl_entropy = getEntropy(data["cl"])

print(cl_entropy)

0.9709505944546686


1.5) Finish the below function for calculating a conditional entropy: $H(CL|X) = - \sum_{x \in X} \sum_{y \in CL} p(x,y) log_2 \frac{p(x,y)}{p(x)}$. Assume that $log_2(0)$ is equal to 0 and if $p(x) = 0$, $\frac{p(x,y)}{p(x)}$ is equal to 0 as well.

In [9]:
def getConditionalEntropy(cl, attr):
    cl = list(cl)
    attr = list(attr)
    templs = [0] * 4
    pcl_0 = list(attr).count(0)/len(attr)
    pcl_1 = 1 - pcl_0
    for i in range(len(cl)):
        if cl[i] == 0 and attr[i] == 0:
            templs[0] += 1
        if cl[i] == 0 and attr[i] == 1:
            templs[1] += 1
        if cl[i] == 1 and attr[i] == 0:
            templs[2] += 1
        if cl[i] == 1 and attr[i] == 1:
            templs[3] +=1
    templs = [item / len(cl) for item in templs]
    conEntropy = 0
    for i in range(2):
        if pcl_0 == 0 or (templs[i]/pcl_0) == 0:
            conEntropy -= 0
        else:
            conEntropy -= templs[i]*math.log((templs[i]/pcl_0),2)
    for i in range(2,4):
        if pcl_1 == 0 or (templs[i]/pcl_1) == 0:
            conEntropy -= 0
        else:
            conEntropy -= templs[i]*math.log((templs[i]/pcl_1),2)
    
    return conEntropy

1.6 ) Calculate conditional entropies for given attribiutes.

In [10]:
print(getConditionalEntropy(data["cl"], data["attr 1"]))
print(getConditionalEntropy(data["cl"], data["attr 5"]))

0.9509775004326937
0.44217935649972373


1.7 ) Which entropy is lesser and why?

1.8) Finish the below function for calculating information gain:

In [11]:
def getInformationGain(cl, attr):
    clEntropy = getEntropy(cl)
    conEntropy = getConditionalEntropy(cl, attr)
    return clEntropy - conEntropy

In [12]:
print(getInformationGain(data["cl"], data["attr 1"]))
print(getInformationGain(data["cl"], data["attr 5"]))

0.01997309402197489
0.5287712379549449


# Part 2: ID3 algorithm

Decision tree consists of decision nodes and leaves. Nodes split data while leaves classify objects. Consider the class "Node" provided below. It consists of 4 fields:
- attr - attribute ID (use the names in attributeNames vector)
- left - left branch, i.e., a reference to other node
- right - right branch, i.e., a reference to other node
- value - a decision. If node = None, then the node is not a leaf. If value is not None, then a node is considered a leaf. 

Method __call__ returns the decision if the node is a leaf (i.e., when value is not None). 
Otherwise, it calls either the left or the right branch of an input object, based on the attribute value (0 -> left children; 1 -> right children). In this way, we can traverse the decision tree in order to find the final decision.

In [13]:
class Node:
    def __init__(self, attr, left, right, value):
        self.attr = attr
        self.left = left
        self.right = right
        self.value = value

    def __call__(self, obj):
        if self.value is None:
            if obj[self.attr] == 0:
                return self.left(obj)
            else:
                return self.right(obj)
        else:
            return self.value
        
### EXAMPLE
def example(obj):
    root = Node(0, None, None, None)
    lChildren = Node(1, None, None, None)
    rChildren = Node(None, None, None, 2)
    root.left = lChildren
    root.right = rChildren
    llChildren = Node(None, None, None, 3)
    rrChildren = Node(None, None, None, 4)
    lChildren.left = llChildren
    lChildren.right = rrChildren
    print(root(obj))
    
example([0, 0])
example([0, 1])
example([1, 0])
example([1, 1])

3
4
2
2


2.1) Create an initial root. Set the value (decision) to 1. 

In [15]:
initialRoot = Node(None, None, None,1)

2.2) Use a getErrorRate method in common.py auxiliary file to calculate the error rate. The decision is made based on the majority rule. In case of tie, the method takes 0 as the default class.

In [16]:
cm.getErrorRate(initialRoot,data)

0.4

2.3) Use printGraph method (see the common.py file) to draw the decision tree and save it in a png file.

In [19]:
cm.printGraph(initialRoot)

2.4) Calculate information gain for all attribiutes.

In [26]:
def informationGain(data,attributeNames):
    gainTab = []
    for item in range(len(attributeNames)):
        gainTab.append(getInformationGain(data["cl"], data[attributeNames[item]]))
    return gainTab
print(informationGain(data,attributeNames))

[0.01997309402197489, 0.16343184481524653, 0.12451124978365313, 0.21351668837532478, 0.5287712379549449]


2.5) Choose the best attribute to split the data. Construct two new nodes: one for $x_i$ = 0 decision and the second for $x_i$ = 1; connect them with the root (left and right branch). Remember to update the root. 

In [30]:
bestAttribiute = attributeNames[informationGain(data,attributeNames).index(max(informationGain(data,attributeNames)))]
init_root = Node(bestAttribiute,None,None,None)
left_node = Node(None,None,None,0)
right_node = Node(None,None,None,1)
init_root.left = left_node
init_root.right = right_node

2.6) Print the graph and calculate the error rate. What happened with the error rate?

In [34]:
cm.printGraph(init_root)
cm.getErrorRate(init_root,data)

0.4

2.7) Split the 'data' (table) based on the selected attribiute, i.e., create two new tables.

In [32]:
left_data = data[data[bestAttribiute] == 0]
right_data = data[data[bestAttribiute] == 1]

2.8) Let us start with the left node. Firstly, calculate information gain for this node.

In [35]:
leftNode = informationGain(left_data,attributeNames)

ZeroDivisionError: division by zero

2.9) Choose the best attribute to split the data and then update the decision tree.

In [ ]:
### TODO

2.10) Print the graph and calculate the error rate. What happened with the error rate?

In [ ]:
### TODO

2.11) Split data (remember that we split left_data, not data).

In [ ]:
### TODO
### leftLeft_data = 
### leftRight_data = 

2.12) Repeat the whole process for the right node.

In [ ]:
# TODO compute the information gain

In [ ]:
# TODO update the decision tree

In [ ]:
# TODO print the decision tree and calculate the error rate

In [ ]:
# TODO split the data (right_data)
#rightLeft_data
#rightRight_data

2.13) Let's consider left-left node. Calculate information gain for it.

In [ ]:
# TODO

2.14) Will adding a new node to the tree improve its effectiveness? Why? Why not?

2.15) Calculate information gain for the left-right node.

In [ ]:
printInformationGain(leftRight_data)

In [ ]:
### Select the attribute and update the tree

In [ ]:
### Print the decision tree and compute the error rate

2.16) What happened with the error rate? Is it necessary to keep these two newly added leaves?

2.17) Finish creating the right side of the tree

In [ ]:
### TODO

# Part 3: automated construction of decision trees

3.1 Complete the following function for automated construct of decision trees, so that it returns a decision tree for the given data and attribute list. Note that this is a recusive method, i.e., calls itself.

In [ ]:
max_depth = 0

def createTree(data, attributeNames, depth=0):
    data = data.reset_index().drop("index", axis=1)
    ### TODO

3.2) Build a decision tree for a training dataset in the common.py auxiliary file, for diffrent values of max_depth.  Calculate & compare the error rates for training and validation datasets.

In [ ]:
max_depth = 10

In [ ]:
# Training dataset
train_attributeNames, train_data = cm.getTrainingDataSet()
### TODO

In [ ]:
# Validation dataset
valid_attributesName, valid_data = cm.getValidationDataSet()
### TODO

3.3) Consider only the training data set and answer the following questions:
* What is the miximum depth of the tree (consider only the training data set)?
* The tree building process should stop when there is no improvement in error rate (why?). Check for which value of "max_dept" there is no improvement in error rate. 

In [ ]:
for i in range(10):
    max_depth = i
    ### TODO